In [1]:
# Lab 9 XOR
import warnings
warnings.simplefilter("ignore")
import tensorflow as tf
import numpy as np

In [2]:
tf.set_random_seed(777)  # for reproducibility

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name="x")
Y = tf.placeholder(tf.float32, [None, 1], name="y")

In [3]:
with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.random_uniform([2, 2]), name="weight_1")
    b1 = tf.Variable(tf.random_uniform([2]), name="bias_1")
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

    tf.summary.histogram("W1", W1)
    tf.summary.histogram("b1", b1)
    tf.summary.histogram("Layer1", layer1)

with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.random_normal([2, 1]), name="weight_2")
    b2 = tf.Variable(tf.random_normal([1]), name="bias_2")
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)

    tf.summary.histogram("W2", W2)
    tf.summary.histogram("b2", b2)
    tf.summary.histogram("Hypothesis", hypothesis)

In [4]:
# cost/loss function
with tf.name_scope("Cost"):
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    tf.summary.scalar("Cost", cost)

with tf.name_scope("Train"):
    train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [5]:
# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
tf.summary.scalar("accuracy", accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [6]:
# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        _, summary, cost_val, w_val = sess.run(
            [train, merged_summary, cost, W1], feed_dict={X: x_data, Y: y_data}
        )
        writer.add_summary(summary, global_step=step)

        if step % 100 == 0:
            print(step, cost_val, "weight:", w_val)

    # Accuracy report
    h, p, a = sess.run(
        [hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data}
    )

    print(f"\nHypothesis:\n{h} \nPredicted:\n{p} \nAccuracy:\n{a}")

0 0.6958813 weight: [[0.56555724 0.14830016]
 [0.4450029  0.7209047 ]]
100 0.6661593 weight: [[0.27992743 1.3479745 ]
 [0.2067321  1.9930536 ]]
200 0.53621197 weight: [[0.7624368 2.8622847]
 [1.4767566 3.6350594]]
300 0.29072398 weight: [[2.3691506 4.0904403]
 [2.483441  4.801646 ]]
400 0.15460843 weight: [[3.2243977 4.848526 ]
 [3.3224382 5.5280056]]
500 0.09570372 weight: [[3.747631  5.3313475]
 [3.8394246 5.99529  ]]
600 0.06595826 weight: [[4.1107764 5.674253 ]
 [4.198565  6.328675 ]]
700 0.048723143 weight: [[4.385569  5.9372616]
 [4.4703355 6.585178 ]]
800 0.037728205 weight: [[4.605516  6.149593 ]
 [4.6878343 6.792762 ]]
900 0.030218298 weight: [[4.788465  6.327252 ]
 [4.8687263 6.966791 ]]
1000 0.024823656 weight: [[4.944924  6.479832 ]
 [5.023406  7.1165056]]
1100 0.020796275 weight: [[5.081559  6.613506 ]
 [5.1584797 7.2478547]]
1200 0.017696973 weight: [[5.2028484 6.732458 ]
 [5.2783766 7.3648868]]
1300 0.01525273 weight: [[5.311932  6.839653 ]
 [5.3862042 7.4704704]]
1400 0

In [7]:
"""
Hypothesis:
[[6.1310326e-05]
 [9.9993694e-01]
 [9.9995077e-01]
 [5.9751470e-05]] 
Predicted:
[[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:
1.0
"""

'\nHypothesis:\n[[6.1310326e-05]\n [9.9993694e-01]\n [9.9995077e-01]\n [5.9751470e-05]] \nPredicted:\n[[0.]\n [1.]\n [1.]\n [0.]] \nAccuracy:\n1.0\n'